In [1]:
import pandas as pd
import json
import numpy as np

In [33]:
test_file = "faa_plmarker.jsonl"
ner_model = "PL-Marker/ace05ner_models/ace05ner-bert-faa"
re_model = "PL-Marker/ace05re_models/ace05re-bert-faa"
data_dir = "ace05"

In [34]:
with open(f"PL-Marker/{data_dir}/{test_file}") as f:
    test_str = f.read()

test = []
for test_doc_str in test_str.split('\n'):
    try:
        test.append(json.loads(test_doc_str))
    except:
        print(f"could not parse: {test_doc_str}")

could not parse: 


In [35]:
len(test[1]['sentences'])

3

In [36]:
idx2word = {}
for idoc in range(len(test)):
    doc = test[idoc]

    idx2word[idoc] = {}
    curr_idx = 0
    for sentence in test[idoc]['sentences']:
        for word in sentence:
            if word not in idx2word[idoc]:
                idx2word[idoc][curr_idx] = word
                curr_idx = curr_idx + 1

In [37]:
with open(f"{ner_model}/ent_pred_test.json") as f:
    ent_pred_str = f.read()

ent_pred = []
for ent_pred_doc_str in ent_pred_str.split('\n'):
    try:
        ent_pred.append(json.loads(ent_pred_doc_str))
    except:
        print(f"could not parse: {ent_pred_doc_str}")

could not parse: 


In [38]:
ent_pred[0].keys()

dict_keys(['sentences', 'ner', 'relations', 'doc_key', 'predicted_ner'])

In [39]:
len(ent_pred[1]['sentences'])

3

In [40]:
def get_span(idoc, sentence, idx0, idx1):
    start_span = sentence.index(idx2word[idoc][idx0])
    end_span = sentence.index(idx2word[idoc][idx1], start_span)
    while True:
        try:
            start_span = sentence.index(idx2word[idoc][idx0], start_span+1, end_span)
        except:
            break
    return sentence[start_span:end_span+1]

In [41]:
for idoc in range(len(ent_pred)):
    print(f"*** Document #{idoc} ****")
    for isentence in range(len(ent_pred[idoc]["sentences"])):
        print(' '.join(ent_pred[idoc]['sentences'][isentence]))
        print("Correct:")
        for entity in ent_pred[idoc]['ner'][isentence]:
            print(entity)
            span = ' '.join(get_span(idoc, ent_pred[idoc]['sentences'][isentence], entity[0], entity[1]))
            print(f"{span} : {entity[2]}")
        print("\nPredicted:")
        for entity in ent_pred[idoc]['predicted_ner'][isentence]:
            print(entity)
            span = ' '.join(get_span(idoc, ent_pred[idoc]['sentences'][isentence], entity[0], entity[1]))
            print(f"{span} : {entity[2]}")
        print("\n----------------\n")

*** Document #0 ****
tailwheel cocked right prior to tkof .
Correct:

Predicted:

----------------

*** Document #1 ****
tow plane became airborne then settled .
Correct:

Predicted:
[1, 1, 'VEH']
plane : VEH

----------------

student thought tow in trouble & released .
Correct:

Predicted:
[7, 7, 'PER']
student : PER

----------------

hit tree .
Correct:

Predicted:

----------------

*** Document #2 ****
2nd ils apch , acft 's g/s inop .
Correct:

Predicted:
[4, 4, 'ORG']
acft : ORG

----------------

lom tuned to wrong freq .
Correct:

Predicted:

----------------

*** Document #3 ****
plt noted soft r brake pedal drg taxi to tkof .
Correct:

Predicted:
[9, 9, 'ORG']
tkof : ORG

----------------

flt rtnd springfield due soft brake strong winds bos
Correct:

Predicted:

----------------

*** Document #4 ****
taxi off hard sfc due tfc r main gr broke through roof of a washed out underground tunnel .
Correct:

Predicted:
[17, 17, 'FAC']
tunnel : FAC

----------------

*** Document #

In [42]:
with open(f"{re_model}/pred_results.json") as f:
    pred_results_str = f.readlines()[0]
    pred_results = json.loads(pred_results_str)

In [43]:
pred_results.keys()

dict_keys(['1', '2', '3', '4', '8', '11', '14', '25', '26', '31', '41', '42', '46', '47', '50', '51', '52', '55', '57', '58', '59', '60', '61', '62', '64', '65', '66', '67', '71', '72', '73', '74', '76', '77', '79', '80', '84', '87', '88', '91', '94', '96', '97', '100', '101', '102', '105', '107', '109', '110', '113', '115', '116', '117', '118', '120', '121', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '137', '139', '140', '142', '145', '146', '148', '149', '150', '151', '152', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '166', '168', '169', '171', '172', '173', '175', '176', '177', '178', '179', '181', '182', '184', '185', '187', '189', '191', '195', '197', '202', '203', '204', '205', '209', '210', '211', '213', '215', '216', '217', '218', '219', '220', '221', '225', '226', '227', '228', '229', '230', '235', '237', '238', '240', '242', '244', '245', '246', '248', '249', '250', '251', '252', '254', '255', '256', '

In [44]:
for doc_str in pred_results.keys():
    idoc = int(doc_str)
    print(f"*** Document #{idoc} ****")
    for isentence in range(len(pred_results[f'{idoc}'])):
        sentence_num = pred_results[f'{idoc}'][isentence][0]
        triples = pred_results[f'{idoc}'][isentence][1]
        sentence = ent_pred[idoc]['sentences'][sentence_num]
        print(' '.join(sentence))
        if len(triples) > 0:
            for triple in triples:
                print(f"\t{' '.join(get_span(idoc, sentence, triple[0][0], triple[0][1]))} --> {triple[2]} --> {' '.join(get_span(idoc, sentence, triple[1][0], triple[1][1]))}")
        else:
            print("\tNo triples found")
        print("\n--------------\n")

*** Document #1 ****
tow plane became airborne then settled .
	No triples found

--------------

student thought tow in trouble & released .
	No triples found

--------------

*** Document #2 ****
2nd ils apch , acft 's g/s inop .
	No triples found

--------------

*** Document #3 ****
plt noted soft r brake pedal drg taxi to tkof .
	No triples found

--------------

*** Document #4 ****
taxi off hard sfc due tfc r main gr broke through roof of a washed out underground tunnel .
	No triples found

--------------

*** Document #8 ****
sufficient opportunity existed to release when glider assumed nose high attitude .
	No triples found

--------------

*** Document #11 ****
plt continued t/o ifr after wndshd & windows fogged up during vfr departure .
	No triples found

--------------

*** Document #14 ****
skid caught under refueling line .
	No triples found

--------------

*** Document #25 ****
plt continued taxiing after total hyd failure .
	No triples found

--------------

*** Documen

In [45]:
results_dict = {"doc_idx":[], "sent_idx":[], "c5_id":[], "original_sentence":[], "input":[], "head":[],"relation":[], "tail":[]}
og_data = pd.read_csv("../../data/FAA_data/Maintenance_Text_data_nona.csv")

for doc_str in pred_results.keys():
    idoc = int(doc_str)
    for isentence in range(len(pred_results[f'{idoc}'])):
        sentence_num = pred_results[f'{idoc}'][isentence][0]
        triples = pred_results[f'{idoc}'][isentence][1]
        sentence = ent_pred[idoc]['sentences'][sentence_num]
        og_sentence = og_data["c119"].iat[idoc]
        c5_id = og_data["c5"].iat[idoc]

        for triple in triples:
            results_dict["doc_idx"].append(idoc)
            results_dict["sent_idx"].append(isentence)
            results_dict["c5_id"].append(c5_id)
            results_dict["original_sentence"].append(og_sentence)
            results_dict["input"].append(' '.join(sentence))
            results_dict["head"].append(' '.join(get_span(idoc, sentence, triple[0][0], triple[0][1])))
            results_dict["relation"].append(triple[2])
            results_dict["tail"].append(' '.join(get_span(idoc, sentence, triple[1][0], triple[1][1])))

In [46]:
for i_triple in range(len(results_dict["doc_idx"])):
    print(f"{results_dict['doc_idx'][i_triple]}, {results_dict['original_sentence'][i_triple]}\t---from---\t{og_data["c119"].iat[results_dict['doc_idx'][i_triple]]}")

46, NEW AIRCRAFT OWNER NOT AWARE OF PURPOSE OR OPERATION OF QUICK DRAIN. PARTIALLY OPENED ON PREFLIGHT.                	---from---	NEW AIRCRAFT OWNER NOT AWARE OF PURPOSE OR OPERATION OF QUICK DRAIN. PARTIALLY OPENED ON PREFLIGHT.                
60, PILOT STRUCK AN UNLIGHTED BARRICADE DURING TAXI FROM LANDING.                                                      	---from---	PILOT STRUCK AN UNLIGHTED BARRICADE DURING TAXI FROM LANDING.                                                      
67, PILOT OVER RAN RUNWAY INTO A DITCH DUE TO BRAKE FAILURE. SYSTEM NEEDED TO BE PURGED OF AIR.                        	---from---	PILOT OVER RAN RUNWAY INTO A DITCH DUE TO BRAKE FAILURE. SYSTEM NEEDED TO BE PURGED OF AIR.                        
91, DOOR POPPED OPEN ON TAKEOFF, PILOT LOST CONTROL WHILE TRYING TO LATCH DOOR, RAN OFF SIDE OF RUNWAY.                	---from---	DOOR POPPED OPEN ON TAKEOFF, PILOT LOST CONTROL WHILE TRYING TO LATCH DOOR, RAN OFF SIDE OF RUNWAY.                
116, PILOT L

In [47]:
pd.DataFrame(results_dict).to_csv(f"../../data/results/pl-marker/ace05_bert_final_results.csv")